In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
cd '/content/drive/MyDrive/Tích hợp dữ liệu'

/content/drive/.shortcut-targets-by-id/1kdSQxLU563_8sd8nvQTr-Z08SiwQ3jyp/Tích hợp dữ liệu


# 1. Import Libraries

In [8]:
import re

In [9]:
!pip install pandas recordLinkage unidecode 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
import json
import uuid

import recordlinkage
from unidecode import unidecode

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt


# 2. Import data

In [11]:
# read data

path_dmx = 'data-sources/dmx.json'
with open(path_dmx) as dmx_json:
    dmx_data = json.load(dmx_json)
dmx_df = pd.DataFrame.from_dict(dmx_data)

path_dmcl = 'data-sources/dmcl.json'
with open(path_dmcl) as dmcl_json:
    dmcl_data = json.load(dmcl_json)
dmcl_df = pd.DataFrame.from_dict(dmcl_data)
    
path_pico = 'data-sources/pico.json'
with open(path_pico) as pico_json:
    pico_data = json.load(pico_json)
pico_df = pd.DataFrame.from_dict(pico_data)

path_nk = 'data-sources/nk.csv'
nk_df = pd.read_csv(path_nk)

In [12]:
# Add index and website
dmx_df['uuid'] = [str(uuid.uuid4()) for _ in range(len(dmx_df.index))]
dmx_df['website'] = 'dmx'
dmcl_df['uuid'] = [str(uuid.uuid4()) for _ in range(len(dmcl_df.index))]
dmcl_df['website'] = 'dmcl'
pico_df['uuid'] = [str(uuid.uuid4()) for _ in range(len(pico_df.index))]
pico_df['website'] = 'pico'
nk_df['uuid'] = [str(uuid.uuid4()) for _ in range(len(nk_df.index))]
nk_df['website'] = 'nk'

### Preprocess ĐMX

In [13]:
# pre-process features of  from list to string
for index, value in enumerate(dmx_df['loai_may']):
    if value == []:
        dmx_df['loai_may'][index] = 'Unknown' 
    else: 
        dmx_df['loai_may'][index] = value[0] 

for index, value in enumerate(dmx_df['hang']):
    if value == []:
        dmx_df['hang'][index] = 'Unknown' 
    else:
        dmx_df['hang'][index] = value[0][:-1]

for index, value in enumerate(dmx_df['khoi_luong_giat']):
    if value == []:
        dmx_df['khoi_luong_giat'][index] = 'Unknown' 
    else:
        dmx_df['khoi_luong_giat'][index] = value[0]

for index, value in enumerate(dmx_df['kich_thuoc_va_khoi_luong']):
    if value == []:
        dmx_df['kich_thuoc_va_khoi_luong'][index] = 'Unknown' 
    else:
        dmx_df['kich_thuoc_va_khoi_luong'][index] = value[0]

dmx_df.head()

,ten,gia,loai_may,khoi_luong_giat,kieu_dong_co,cong_nghe_giat,tien_ich,bang_dieu_khien,chat_lieu,kich_thuoc_va_khoi_luong,hang,uuid,website
0,Máy giặt Panasonic 9 Kg NA-F90A9BRV,7.120.000₫,Cửa trên,9 Kg,[Truyền động gián tiếp (dây Curoa)],"[Công nghệ giặt chuyên biệt StainMaster, Công ...","[Bộ lọc sơ vải Ag, Hẹn giờ giặt, Khóa trẻ em, ...",[Song ngữ Anh - Việt có nút nhấn],[Thép không gỉ],Cao 105.3 cm - Ngang 54.5 cm - Sâu 61 cm - Nặn...,Panasonic.,a8ed6645-4385-44bf-bb15-df14e596ba56,dmx
1,Máy giặt Toshiba Inverter 8.5 Kg TW-BH95S2V WK,7.990.000₫,Cửa trước,8.5 Kg,[Truyền động gián tiếp (dây Curoa)],"[Giặt nước nóng, Greatwaves sức mạnh siêu sóng...","[Hẹn giờ giặt, Khóa trẻ em, Tự khởi động lại k...","[Song ngữ Anh - Việt có nút xoay, nút nhấn và ...",[Thép không gỉ],Cao 84.5 cm - Ngang 59.5 cm - Sâu 60 cm - Nặng...,Toshiba.,7db5eccc-0abd-4060-8afe-d5cfb0f9a77b,dmx
2,Máy giặt Panasonic 8.5 Kg NA-F85A9BRV,6.190.000₫,Cửa trên,8.5 Kg,[Truyền động gián tiếp (dây Curoa)],"[Công nghệ giặt chuyên biệt StainMaster, Công ...","[Bộ lọc sơ vải Ag, Hẹn giờ giặt, Khóa trẻ em, ...",[Song ngữ Anh - Việt có nút nhấn],[Thép không gỉ],Cao 104.3 cm - Ngang 54.6 cm - Sâu 61 cm - Nặn...,Panasonic.,131195fc-c405-4459-aba5-5b69c1edccc1,dmx
3,Máy giặt LG Inverter 9 kg FV1409S4W,11.290.000₫,Cửa trước,9 Kg,[Truyền động trực tiếp - sử dụng trí tuệ nhân ...,"[Công nghệ AI DD bảo vệ sợi vải, Công nghệ giặ...",[Cho phép điều khiển máy giặt từ xa qua ứng dụ...,"[Song ngữ Anh - Việt, có nút xoay, màn hình hi...",[Thép không gỉ],Cao 85 cm - Ngang 60 cm - Sâu 63 cm - Nặng 62 kg,LG.,63380186-80b6-40a7-bab1-bda028755307,dmx
4,Máy giặt Samsung Inverter 10 kg WA10T5260BY/SV,7.590.000₫,Cửa trên,10 Kg,[Truyền động trực tiếp bền & êm],"[Bộ lọc xơ vải Magic Filter, Chế độ giặt lưu h...",[Chẩn đoán sự cố thông minh qua ứng dụng điện ...,[Song ngữ Anh - Việt nút nhấn có màn hình hiển...,[Thép không gỉ],Cao 107.5 cm - Ngang 61 cm - Sâu 67.5 cm - Nặn...,Samsung.,48be9d7f-81ba-41fe-8cb3-0033d6f1c104,dmx


In [14]:
dmx_df.hang = dmx_df.hang.apply(lambda x: x.split(".")[0].lower())
dmx_df.khoi_luong_giat = dmx_df.khoi_luong_giat.apply(lambda x: x.split(" ")[0])
dmx_df.head()

,ten,gia,loai_may,khoi_luong_giat,kieu_dong_co,cong_nghe_giat,tien_ich,bang_dieu_khien,chat_lieu,kich_thuoc_va_khoi_luong,hang,uuid,website
0,Máy giặt Panasonic 9 Kg NA-F90A9BRV,7.120.000₫,Cửa trên,9,[Truyền động gián tiếp (dây Curoa)],"[Công nghệ giặt chuyên biệt StainMaster, Công ...","[Bộ lọc sơ vải Ag, Hẹn giờ giặt, Khóa trẻ em, ...",[Song ngữ Anh - Việt có nút nhấn],[Thép không gỉ],Cao 105.3 cm - Ngang 54.5 cm - Sâu 61 cm - Nặn...,panasonic,a8ed6645-4385-44bf-bb15-df14e596ba56,dmx
1,Máy giặt Toshiba Inverter 8.5 Kg TW-BH95S2V WK,7.990.000₫,Cửa trước,8.5,[Truyền động gián tiếp (dây Curoa)],"[Giặt nước nóng, Greatwaves sức mạnh siêu sóng...","[Hẹn giờ giặt, Khóa trẻ em, Tự khởi động lại k...","[Song ngữ Anh - Việt có nút xoay, nút nhấn và ...",[Thép không gỉ],Cao 84.5 cm - Ngang 59.5 cm - Sâu 60 cm - Nặng...,toshiba,7db5eccc-0abd-4060-8afe-d5cfb0f9a77b,dmx
2,Máy giặt Panasonic 8.5 Kg NA-F85A9BRV,6.190.000₫,Cửa trên,8.5,[Truyền động gián tiếp (dây Curoa)],"[Công nghệ giặt chuyên biệt StainMaster, Công ...","[Bộ lọc sơ vải Ag, Hẹn giờ giặt, Khóa trẻ em, ...",[Song ngữ Anh - Việt có nút nhấn],[Thép không gỉ],Cao 104.3 cm - Ngang 54.6 cm - Sâu 61 cm - Nặn...,panasonic,131195fc-c405-4459-aba5-5b69c1edccc1,dmx
3,Máy giặt LG Inverter 9 kg FV1409S4W,11.290.000₫,Cửa trước,9,[Truyền động trực tiếp - sử dụng trí tuệ nhân ...,"[Công nghệ AI DD bảo vệ sợi vải, Công nghệ giặ...",[Cho phép điều khiển máy giặt từ xa qua ứng dụ...,"[Song ngữ Anh - Việt, có nút xoay, màn hình hi...",[Thép không gỉ],Cao 85 cm - Ngang 60 cm - Sâu 63 cm - Nặng 62 kg,lg,63380186-80b6-40a7-bab1-bda028755307,dmx
4,Máy giặt Samsung Inverter 10 kg WA10T5260BY/SV,7.590.000₫,Cửa trên,10,[Truyền động trực tiếp bền & êm],"[Bộ lọc xơ vải Magic Filter, Chế độ giặt lưu h...",[Chẩn đoán sự cố thông minh qua ứng dụng điện ...,[Song ngữ Anh - Việt nút nhấn có màn hình hiển...,[Thép không gỉ],Cao 107.5 cm - Ngang 61 cm - Sâu 67.5 cm - Nặn...,samsung,48be9d7f-81ba-41fe-8cb3-0033d6f1c104,dmx


### Preprocess ĐMCL

In [16]:
dmcl_df.hang = dmcl_df.hang.apply(lambda x: x.split(" ")[0].lower())
def f(row):
    if row['loai_may'] == "Lồng đứng":
        val = 'cửa trên'
    else:
        val = 'cửa trước'
    return val


dmcl_df['loai_may_cua'] = dmcl_df.apply(f, axis=1)
dmcl_df.khoi_luong_giat = dmcl_df.khoi_luong_giat.astype(str)
dmcl_df.khoi_luong_giat = dmcl_df.khoi_luong_giat.apply(lambda x: x.split(" ")[0])


In [41]:
dmcl_df.khoi_luong_giat = dmcl_df.khoi_luong_giat.apply(lambda x: re.sub('Kg', '',x.split(" ")[0]))
dmcl_df.khoi_luong_giat = dmcl_df.khoi_luong_giat.apply(lambda x: re.sub(',', '.',x))


In [42]:
dmcl_df.head(20)

,ten,gia,loai_may,khoi_luong_giat,cong_nghe_giat,bang_dieu_khien,chat_lieu,kich_thuoc,hang,uuid,website,loai_may_cua
0,Máy Giặt Panasonic 9 Kg NA-F90A9DRV,7.290.000,Lồng đứng,9.0,"Giặt chuyên biệt StainMaster , Mâm giặt kháng ...",Nút nhấn,Thép không gỉ,554 x 617 x 1.043 mm,panasonic,f736e62c-0648-4572-8031-174b237eeb16,dmcl,cửa trên
1,Máy Giặt LG Inverter 11 Kg FV1411S3B,13.990.000,Lồng ngang,11,"Công nghệ AI DD bảo vệ sợi vải, Công nghệ giặt...","Có nút xoay, màn hình hiển thị",Thép không gỉ,58.2 x 58 x 84 cm,lg,468f6b79-6cb1-45c3-9dfe-aad57faa3c5c,dmcl,cửa trước
2,Máy Giặt Candy Inverter 8 Kg 1284DWH7 / 1-S,5.390.000,Lồng ngang,8,NaN,NaN,Thép không gỉ,600 x 520 x 850cm,candy,85d79148-c4ee-40ae-a3e2-03e77b619a87,dmcl,cửa trước
3,Máy Giặt LG 11 Kg TH2111SSAB,9.290.000,Lồng đứng,11,"Công nghệ giặt hơi nước Steam, Turbowash 3D tí...",Cảm ứng,Thép không gỉ,540 x 540 x 945mm,lg,f7dbd642-039a-45d4-a974-b97d3b706d79,dmcl,cửa trên
4,Máy Giặt Panasonic 10.5 Kg NA-FD10XR1LV,9.290.000,Lồng đứng,10.5,"Giặt chuyên biệt StainMaster, Xoáy nước siêu m...",Nút nhấn,Thép không gỉ,724 x 639 x 1049mm,panasonic,0fad31fd-5bdf-4331-801b-e7baa1ac6d0b,dmcl,cửa trên
5,Máy Giặt Samsung Inverter 8.5 Kg WW85T4040CE /...,8.490.000,Lồng ngang,8.5,Giặt hơi nước diệt khuẩn Hygiene Steam giúp lo...,Nút xoay chọn chương trình,Thép không gỉ,NaN,samsung,53f00e33-d77b-4703-b5c2-a9e3bac031ba,dmcl,cửa trước
6,Máy Giặt Aqua 10 Kg AQW-DR101GT BK,7.290.000,Lồng đứng,10,"Lồng giặt Pillow, Mâm giặt kháng khuẩn ABT",Bảng điều khiển phía sau và núm xoay tiện dụng,Thép không gỉ,560 x 630 x 1035 mm,aqua,8169e616-f265-4404-a5cd-f29989c0ef0a,dmcl,cửa trên
7,Máy Giặt Bosch 10 Kg WGG254A0SG,9.290.000,Lồng ngang,10,Công nghệ VarioPerfectTM,NaN,Thép không gỉ,Cao 84.8 cm x Rộng 59.8 cm x Sâu 59.0 cm,bosch,ebe73437-baaf-4bbc-b479-c3ca07a646c5,dmcl,cửa trước
8,Máy Giặt Samsung Inverter AI 9 Kg WW90TP44DSH ...,9.290.000,Lồng ngang,9.0,"AI Wash tự động tối ưu lượng nước giặt xả, lượ...",Nút xoay chọn chương trình,Thép không gỉ,NaN,samsung,35dfc992-b4da-4f1c-b650-be88c9e3158f,dmcl,cửa trước
9,Máy Giặt Toshiba 10.5 Kg TW-BK115G4V (SS),13.390.000,Lồng ngang,10.5,"Greatwave sức mạnh siêu sóng, Chế độ giặt nhan...",Nút xoay chọn chương trình,Thép không gỉ,595 x 615 x 850mm,toshiba,ebbc2970-3688-42a7-8725-a8b738ee1a9f,dmcl,cửa trước


### Preprocess pico

In [18]:
pico_df.hang = pico_df.hang.astype(str)
pico_df.hang = [x.lower() for x in pico_df.hang]
pico_df.khoi_luong_giat = pico_df.khoi_luong_giat.astype(str)

pico_df.khoi_luong_giat = pico_df.khoi_luong_giat.apply(lambda x: re.sub('kg', '',x.split(" ")[0]))

In [19]:
pico_df.head()

,ten,gia,loai_may,khoi_luong_giat,tien_ich,bang_dieu_khien,kich_thuoc,hang,uuid,website
0,Máy Giặt Panasonic 8kg NAF80VS9GRV,5.290.000₫,Panasonic,8.0,11 chế độ,Tiếng Việt,616 x 696 x 1053 (R*S*C) mm,panasonic,1487dda0-1362-40e6-8d2d-5464e8759881,pico
1,Máy Giặt Sấy Electrolux inverter EWW1141AEWA -...,17.490.000₫,Máy Giặt Sấy,11.0,NaN,NaN,600 x 669 x 850 mm,electrolux,e4b5e2ed-7d2c-4f18-acf9-59f13224e0f3,pico
2,Máy Giặt Samsung WW10K54E0UW/SV 10kg Inverter,11.590.000₫,Máy giặt cửa trước,10.0,Công nghệ giặt bong bóng Eco Bubble,Bảng điều khiển : Song ngữ Anh - Việt có nút xoay,550 x600 x850 mm,samsung,5e5af3d6-c0f9-4f3c-8245-596292dc426d,pico
3,Máy Giặt Aqua DD Inverter 8.5 kg AQD-D850E W M...,6.390.000₫,Máy giặt cửa trước,8.5,16 Chương trình giặt,Tiếng Việt,595 x 525 x 850 (R x S x C) (mm),aqua,2f11b6f4-e141-4e70-bf85-3a95ef5e066f,pico
4,Máy Giặt Aqua AQD-DD950E.S DD Inverter 9.5 Kg,8.990.000₫,Cửa trước,9.5,16 chương trình,Tiếng Việt,595 x 595 x 850mm,aqua,f426829d-bdd7-4490-9a3b-7dcd296402e7,pico


### Preprocess NK

In [20]:
nk_df["Nhà sản xuất:"] = nk_df["Nhà sản xuất:"].apply(lambda x: x.lower())
nk_df["Khối lượng quần áo:"] = nk_df["Khối lượng quần áo:"].astype(str)
nk_df["title"] = nk_df["title"].astype(str)

def f2(row):
  val = row["title"]
  pattern = re.search("\d+?\s", val)
  if (pattern == None):
    return "NaN"
  else:
    return pattern[0][:-1]

nk_df["khoi_luong_giat"] = nk_df.apply(f2, axis=1)

In [21]:
nk_df.head()

,title,price,Model:,Màu sắc:,Nhà sản xuất:,Xuất xứ:,Năm ra mắt :,Thời gian bảo hành:,Địa điểm bảo hành:,Công suất:,Khối lượng quần áo:,Bảng điều khiển:,Chế độ giặt:,Loại máy giặt:,Khối lượng sản phẩm (kg):,Kích thước sản phẩm:,uuid,website,khoi_luong_giat
0,Tủ chăm sóc quần áo thông minh LG Styler S3WF ...,37.000.000đ,S3WF,Trắng,lg,Hàn Quốc,2019.0,12 tháng,Nguyễn Kim,1500 W,5.2 kg,"Màn hình LCD, phím bấm cảm ứng","Chế độ Refresh: Khử mùi, làm mới quần áo; San...",Styler,60 kg,445 × 1850 × 585 mm,891c20a5-476d-473b-910f-916c620c19e4,nk,NaN
1,Máy giặt Samsung Inverter 10 kg WW10TP44DSB,13.690.000đ,WW10TP44DSB,Đen,samsung,Việt Nam,2021.0,24 Tháng,Nguyễn Kim,NaN,nan,NaN,Nhiều chế độ giặt,Cửa trước,68 kg,600 x 850 x 550 mm,7ca4c23d-55d7-4ea9-b76a-5c89ebfc3d3c,nk,10
2,Máy giặt Panasonic Inverter 14 kg NA-FD14V1BRV,17.090.000đ,NA-FD14V1BRV,Đen,panasonic,Việt Nam,2021.0,2 năm/ 12 năm động cơ,Nguyễn Kim,NaN,nan,NaN,"10 chương trình (có giặt thường, giặt nhanh, ...",Cửa trên,51 kg,554 × 618 × 1050 mm,14fef901-c75a-43eb-8894-d68cd6508f91,nk,14
3,Máy giặt Samsung Inverter 12 kg WW12TP94DSB,19.790.000đ,WW12TP94DSB,Đen,samsung,Việt Nam,2021.0,24 Tháng,Nguyễn Kim,NaN,nan,NaN,Nhiều chế độ,Cửa trước,76 kg,600 x 850 x 650 mm,122aca92-f0ac-4f73-a860-3eaa3cbe2c52,nk,12
4,Máy giặt Panasonic Inverter 9 kg NA-V90FX1LVT,12.890.000đ,NA-V90FX1LVT,Xám,panasonic,Việt Nam,2019.0,"2 năm, 12 năm động cơ",Nguyễn Kim,1740W,nan,NaN,"15 chương trình (có giặt đồ cotton, giặt hàng...",Cửa trước,75 kg,600 × 596 × 845 mm,afa7d953-1902-4aee-aa8f-79937631e39b,nk,9


# Grouping data 

In [43]:
# DMX
dmx_schema = dmx_df[['uuid', 'ten', 'gia', 'hang', 'loai_may', 'khoi_luong_giat', 'website']]

dmx_mapping = {
    'uuid':'id',
    'ten':'name',
    'gia':'price',
    'hang':'brand',
    'loai_may':'type',
    'khoi_luong_giat': 'load_size',
    'website':'website',
}

# DMCL
dmcl_schema = dmcl_df[['uuid', 'ten', 'gia', 'hang', 'loai_may_cua', 'khoi_luong_giat', 'website']]

dmcl_mapping = {
    'uuid': 'id',
    'ten': 'name',
    'gia': 'price',
    'hang':'brand',
    'loai_may_cua':'type',
    'khoi_luong_giat':'load_size',
    'website': 'website',
}

# Pico
pico_schema = pico_df[['uuid', 'ten', 'gia', 'hang', 'loai_may', 'khoi_luong_giat', 'website']]

pico_mapping = {
    'uuid': 'id',
    'ten': 'name',
    'gia': 'price',
    'hang': 'brand',
    'loai_may': 'type',
    'khoi_luong_giat':'load_size',
    'website':'website'
}

# Nguyen KIm
nk_schema = nk_df[['uuid', 'title', 'price', 'Nhà sản xuất:', 'Loại máy giặt:', 'khoi_luong_giat', 'website']]

In [ ]:
mediated_features = ['id', 'name', 'price', 'brand', 'type', 'load_size', 'website']


# 4. Test recordlinkage

In [ ]:
dmx_schema.shape

(185, 7)

In [ ]:
dmcl_schema[:185].shape

(185, 7)

In [ ]:
import sys
sys.setrecursionlimit(3000)

In [ ]:
indexer = recordlinkage.Index()
indexer.block('uuid')
candidate_links = indexer.index(dmx_schema[:10], dmcl_schema[:20])

c = recordlinkage.Compare()
c.string('ten', 'ten', method='jarowinkler', threshold=0.7)
c.exact('hang', 'hang')
c.string('loai_may', 'loai_may_cua', method='jarowinkler', threshold=0.8)
c.numeric('khoi_luong_giat', 'khoi_luong_giat', method='gauss')

feature_vectors = c.compute(candidate_links, dmx_schema[:10], dmcl_schema[:20])

ecm = recordlinkage.ECMClassifier()
ecm.fit_predict(feature_vectors)

In [23]:
!pip install strsimpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 45 kB 2.1 MB/s 


In [24]:
from strsimpy.jaro_winkler import JaroWinkler

jw = JaroWinkler()

In [63]:
def name_sim(name1, name2):
  #print (jw.similarity(name1, name2))
  return jw.similarity(name1, name2)

def type_sim(type1, type2):
  if (type1.lower() == type2.lower()):
    return 1
  else:
    return 0

def brand_sim(brand1, brand2):
  if (brand1 == brand2):
    return 1
  else: 
    return 0

def load_sim(load1, load2):
  try: 
    load1 = float(load1)
  except:
    return 0
  try: 
    load2 = float(load2)
  except:
    return 0
  if abs(load2 - load1) > 2.0:
    return 0
  else:
    return 1 - abs(load2 - load1) / 2.0
  

sim_id = []
sim_idx = []
for org_idx in range(0, dmx_schema.shape[0]):
  record_origin = dmx_schema.loc[org_idx]
  id1, name1, brand1, type1, load1 = record_origin[0], record_origin[1], record_origin[3], record_origin[4], record_origin[5]
  max_sim = 0
  id = 0
  idx = org_idx
  #print (org_idx)
  for comp_idx in range(0, nk_schema.shape[0]):
    record_comp = dmcl_schema.loc[comp_idx]
    id2, name2, brand2, type2, load2 = record_comp[0], record_comp[1], record_comp[3], record_comp[4], record_comp[5]
    sim = 0.3*brand_sim(brand1, brand2) + 0.3*type_sim(type1, type2) + 0.2*name_sim(name1, name2) + 0.2*load_sim(load1, load2)
    if sim > max_sim:
      max_sim = sim
      id = id2
      idx = comp_idx

  sim_id.append(id)
  sim_idx.append(idx)

In [60]:
my_df = pd.DataFrame(list(zip(dmx_schema["uuid"], sim_id)), columns=["dmx", "dmcl"])

In [62]:
my_df["pico"] = sim_id

In [64]:
my_df["nk"] = sim_id

In [67]:
my_df.to_csv("match_id.csv", index=False)

In [ ]:
dmx_schema.loc[0]

uuid               0f89c033-720f-42fb-ac97-ad7dab720b8a
ten                 Máy giặt Panasonic 9 Kg NA-F90A9BRV
gia                                         7.120.000₫ 
hang                                          panasonic
loai_may                                       Cửa trên
khoi_luong_giat                                       9
website                                             dmx
Name: 0, dtype: object

In [ ]:
index = [0, 11, 13, 18, 86, 139, 183, 184, 191]
for i in index:
  print(dmcl_schema.loc[i])

uuid               ae3237f2-5200-40dd-9e2d-fd3e549d71a8
ten                Máy Giặt Panasonic 9 Kg NA-F90A9DRV 
gia                                           7.290.000
hang                                          panasonic
loai_may_cua                                   cửa trên
khoi_luong_giat                                     9.0
website                                            dmcl
Name: 0, dtype: object
uuid                 5daf380e-da43-4e5d-9727-4edd0c42356c
ten                Máy Giặt Panasonic 10 Kg NA-F100A9DRV 
gia                                             7.990.000
hang                                            panasonic
loai_may_cua                                     cửa trên
khoi_luong_giat                                      10.0
website                                              dmcl
Name: 11, dtype: object
uuid                 3e5d446f-3de8-4832-97ba-bd3682b337a4
ten                Máy Giặt Panasonic 10 Kg NA-F10S10BRV 
gia                                    

In [ ]:
dmx_schema

,uuid,ten,gia,hang,loai_may,khoi_luong_giat,website
0,0f89c033-720f-42fb-ac97-ad7dab720b8a,Máy giặt Panasonic 9 Kg NA-F90A9BRV,7.120.000₫,panasonic,Cửa trên,9,dmx
1,f5dceeb2-5730-4e5e-9f03-b2ff3adb272a,Máy giặt Toshiba Inverter 8.5 Kg TW-BH95S2V WK,7.990.000₫,toshiba,Cửa trước,8.5,dmx
2,7b045833-1ba5-4a0c-bfb7-c8e489ea486e,Máy giặt Panasonic 8.5 Kg NA-F85A9BRV,6.190.000₫,panasonic,Cửa trên,8.5,dmx
3,beca82fe-43f7-4f85-9a41-26de1628594c,Máy giặt LG Inverter 9 kg FV1409S4W,11.290.000₫,lg,Cửa trước,9,dmx
4,d2705ea7-b201-405c-8826-eba762041470,Máy giặt Samsung Inverter 10 kg WA10T5260BY/SV,7.590.000₫,samsung,Cửa trên,10,dmx
...,...,...,...,...,...,...,...
180,b3fd8f1c-2e08-42a9-b53b-41c1a8e6921a,Máy giặt sấy Whirlpool Inverter 9.5 kg WWEB95...,16.490.000₫,whirlpool,Máy giặt sấy,Giặt,dmx
181,d1419c37-cc0e-4393-ac86-b82701ad41bd,Máy giặt Whirlpool Inverter 9.5 kg VWVD9502FG,10.490.000₫,whirlpool,Cửa trên,9.5,dmx
182,72c3b334-d3d5-4ab0-829c-8de9bf339787,Máy giặt Whirlpool 10.5 kg VWVC10502FS,9.490.000₫,whirlpool,Cửa trên,10.5,dmx
183,1e3a7a29-4524-4213-aba6-1b67a85e769c,Máy giặt Whirlpool Inverter 10 kg VWIID1002FG,11.490.000₫,whirlpool,Cửa trên,10,dmx


In [ ]:
dmx_schema.loc[10]

uuid                            c06ad077-c573-44ad-b300-ae1262470864
ten                 Máy giặt Toshiba Inverter 9.5 Kg TW-BK105S3V(SK)
gia                                                     11.990.000₫ 
hang                                                         toshiba
loai_may                                                   Cửa trước
khoi_luong_giat                                                  9.5
website                                                          dmx
Name: 10, dtype: object

In [ ]:
dmcl_schema.loc[13]

uuid                 3e5d446f-3de8-4832-97ba-bd3682b337a4
ten                Máy Giặt Panasonic 10 Kg NA-F10S10BRV 
gia                                             6.300.000
hang                                            panasonic
loai_may_cua                                     cửa trên
khoi_luong_giat                                        10
website                                              dmcl
Name: 13, dtype: object